In [1]:
%matplotlib qt
import numpy as np
import matplotlib.pyplot as plt
import hyperspy.api as hs

In [2]:
import diffpy
from diffsims.generators.rotation_list_generators import get_beam_directions_grid
from diffsims.generators.diffraction_generator import DiffractionGenerator
from diffsims.libraries.structure_library import StructureLibrary
from diffsims.generators.library_generator import DiffractionLibraryGenerator
from pyxem.utils.calibration_utils import find_diffraction_calibration

In [16]:
dp = hs.signals.Signal2D(np.random.rand(20,10,30,30))

In [17]:
dp

<Signal2D, title: , dimensions: (10, 20|30, 30)>

In [21]:
def default_structure():
    """An atomic structure represented using diffpy"""
    latt = diffpy.structure.lattice.Lattice(3, 3, 5, 90, 90, 120)
    atom = diffpy.structure.atom.Atom(atype="Ni", xyz=[0, 0, 0], lattice=latt)
    hexagonal_structure = diffpy.structure.Structure(atoms=[atom], lattice=latt)
    return hexagonal_structure

In [19]:
from diffsims.generators.rotation_list_generators import get_beam_directions_grid
resolution = 1 # maximum angle in degrees between nearest templates. Pretty rough grid for speed.
grid_cub = get_beam_directions_grid("cubic", resolution, mesh="spherified_cube_edge")
print("Number of patterns: ", grid_cub.shape[0])
# Parameters necessary for simulating a template library
diffraction_calibration = 0.009381971999999999
# half size of the images
half_shape = (dp.data.shape[-2]//2, dp.data.shape[-1]//2)
# maximum radius in reciprocal space to calculate spot intensities for
reciprocal_radius = np.sqrt(half_shape[0]**2 + half_shape[1]**2)*diffraction_calibration


Number of patterns:  1081


In [33]:
# importing the structures
structure_matrix = default_structure()
# structure_matrix = diffpy.structure.loadStructure("Data/Si_mp-149_conventional_standard.cif")
# structure_matrix = diffpy.structure.loadStructure("Data/Si_sudofcc.cif")
# "The microscope"
diff_gen = DiffractionGenerator(accelerating_voltage=200,
                                precession_angle=1,
                                scattering_params=None,
                                shape_factor_model="linear",
                                minimum_intensity=0.1,
                                )

lib_gen = DiffractionLibraryGenerator(diff_gen)

# Generating a library
# "Library of structures and orientations"
library_phases = StructureLibrary(["Cu"], [structure_matrix], [grid_cub])
# Calculate the actual library

diff_lib = lib_gen.get_diffraction_library(library_phases,
                                           calibration=diffraction_calibration,
                                           reciprocal_radius=reciprocal_radius,
                                           half_shape=half_shape,
                                           with_direct_beam=False,
                                           max_excitation_error=0.06)

In [34]:
from pyxem.utils import indexation_utils as iutls

In [35]:
frac_keep = 1
n_best = 15
n_keep = None

delta_r = 1
delta_theta = 1
max_r = None
intensity_transform_function = None
direct_beam_position = None
normalize_image = False
normalize_templates = True

result, phasedict = iutls.index_dataset_with_template_rotation(dp,
                                                    diff_lib,
                                                    phases = ["Cu"],
                                                    n_best = n_best,
                                                    frac_keep = frac_keep,
                                                    n_keep = n_keep,
                                                    delta_r = delta_r,
                                                    delta_theta = delta_theta,
                                                    max_r = None,
                                                    intensity_transform_function=intensity_transform_function,
                                                    normalize_images = normalize_image,
                                                    normalize_templates=normalize_templates,
                                                    target='cpu'
                                                    )

[                                        ] | 0% Completed |  0.1s

/home/joseph/miniconda3/envs/pyxem14/lib/python3.8/site-packages/pyxem/utils/indexation_utils.py:668: RuntimeWarning: invalid value encountered in true_divide
  return array / norms[:, None]


[########################################] | 100% Completed |  0.3s


In [36]:
result['orientation'].shape

(20, 10, 15, 3)

In [37]:
xmap = iutls.results_dict_to_crystal_map(result,phasedict)
xmap.phases[0].space_group = 225

In [38]:
xmap.shape

(20, 10)

In [40]:
from orix import plot
from orix.crystal_map import CrystalMap, Phase, PhaseList
from orix.io import load, save
from orix.quaternion import Orientation, Rotation, symmetry
from orix.vector import Vector3d

In [41]:
ckey_m3m = plot.IPFColorKeyTSL(xmap.phases["Cu"].point_group, direction=Vector3d.yvector())
rgb_cu = ckey_m3m.orientation2color(xmap["Cu"].orientations)

In [42]:
xmap.plot(rgb_cu)